In [1]:
import os
import luigi
import datetime
from google.cloud import bigquery, storage
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import time
import uuid

private_key = 'serviceaccount.json'
# output_path = folder
bq_client = bigquery.Client()#.from_service_account_json(private_key)
storage_client = storage.Client()
project_id = 'projectidtest'

/Users/thesor/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/thesor/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/doc

## upload csv to BQ thru GCS

In [ ]:
pwd

In [ ]:
project_id = 'projectidtest'
dataset_id = 'dw_data'
destination_table_id = 'nvc_w_pii'
bucket_name = 'bi_poc'
source_file_name = './nvc_w_pii.csv'
destination_blob_name = dataset_id + '/' + destination_table_id + '.csv'

# Field types: https://cloud.google.com/bigquery/docs/reference/rest/v2/tables#tablefieldschema
table_schema = [
    bigquery.SchemaField('account_id', 'STRING'),
    bigquery.SchemaField('first_name', 'STRING'),
    bigquery.SchemaField('last_name', 'STRING'),
    bigquery.SchemaField('account_name', 'STRING'),
    bigquery.SchemaField('operator', 'STRING'),
    bigquery.SchemaField('validation_ip', 'STRING'),
    bigquery.SchemaField('ndc', 'STRING'),
    bigquery.SchemaField('affiliate_type', 'STRING'),
    bigquery.SchemaField('affiliate_id', 'STRING'),
    bigquery.SchemaField('validation_date', 'STRING'),
]





"""
Uploads a file to the bucket.
https://cloud.google.com/storage/docs/uploading-objects
"""
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

print('Uploaded file {} to {}.'.format(source_file_name, destination_blob_name))





"""
Load CSV from GCS to BQ
https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv
"""
dataset_ref = bq_client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.schema = table_schema
job_config.skip_leading_rows = 1
job_config.max_bad_records = 100
job_config.field_delimiter = ","
job_config.source_format = bigquery.SourceFormat.CSV # The source format defaults to CSV, so this line is optional.
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE # https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv#loading_csv_data_with_schema_auto-detection
job_config.allow_quoted_newlines = True

uri = 'gs://' + bucket_name + '/' + destination_blob_name

load_job = bq_client.load_table_from_uri(
    uri,
    dataset_ref.table(destination_table_id),
    job_config=job_config)  # API request
print('Starting job {} from {}'.format(load_job.job_id, uri))

load_job.result()  # Waits for table load to complete.
print('Job finished.')

destination_table = bq_client.get_table(dataset_ref.table(destination_table_id))
print('Resulted table has {} rows.'.format(destination_table.num_rows))

In [ ]:
load_job.errors

## list tables

In [40]:
dataset_ref = bq_client.dataset('mrgreen')

tables = list(bq_client.list_tables(dataset_ref))

In [ ]:
table_list_refs = [o.reference for o in tables if o.table_id.startswith('fast_')]
print(len(table_ids))
table_list_refs[:5]

## delete tables

In [ ]:
# for t in table_list_refs:
#     bq_client.delete_table(t)
#     print('Table {}:{} deleted.'.format(dataset_id, t))

In [ ]:
dataset_id = 'mrgreen'
table_id = 'test2'
table_ref = bq_client.dataset(dataset_id).table(table_id)
table_path = table_ref.path
num_rows = bq_client.get_table(table_ref).num_rows
bq_client.delete_table(table_ref)
print('Table {} deleted. {} Rows'.format(table_path, num_rows))

In [ ]:
table_ref.path

In [3]:
df = pd.read_excel('nvc_w_pii.xlsx')
df.to_csv('nvc_w_pii.csv', encoding='utf-8', index=False)